# Graph Structure
The coupling $J_{i,j}(\sigma_i,\sigma_j)$ will be take into account only if an edge exists between site $i$ and $j$.

In [ ]:
%load_ext cython
%config IPCompleter.greedy=True 
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

N = 200  # Number of nodes in the graph
p = 0.02  # Probability that two nodes are connected 
graph = nx.erdos_renyi_graph(N, p)
layout = nx.drawing.layout.fruchterman_reingold_layout(graph)  # For having the same layout at each drawing
nx.draw(graph, with_labels=True, pos=layout)

# Generation of data for variables belonging to more than two states
In the case of the Potts model :
$$
P(\sigma_1,.,\sigma_N) = \frac{\exp\left[\frac{1}{T}\left({\sum_{i=1}^N  h_i(\sigma_i) + \sum_{i,j} J_{ij}(\sigma_i,\sigma_j)}\right) \right]}{ Z}
$$
What I need to generate data is:
$$
J_{ij}(\sigma_i,\sigma_j)  \text{ and } h_i(\sigma_i) 
$$
The Metropolis--Hastings algorithm implies:
$$
P_{a\rightarrow b} = \min\left\{1, \frac{\pi(b)}{\pi(a)}\right\} = \min\left\{1,\exp \left[ \frac{1}{T} \left({{\sum_{i=1}^N  h_i(\sigma_i^b)- h_i(\sigma_i^a) + \sum_{i,j} J_{ij}(\sigma_i^b,\sigma_j^b) - J_{ij}(\sigma_i^a,\sigma_j^a)}}\right)\right] \right\}
$$
where $P_{a\rightarrow b}$ is the transition probability in the MCMC algorithm and $\pi(a)$ the stationary probability of being in the state $b$, i.e $P(\sigma_1,.,\sigma_N)$.

IMPORTANT :
Choice of Ising Gauge : 
$$
\sum_{\sigma_i} J_{ij}(\sigma_i,\sigma_j) = \sum_{\sigma_j} J_{ij}(\sigma_i,\sigma_j) = 0
$$
To go in this Gauge, we infer $J_{ij}$ with $J_{ij} = C^{-1}_{ij}$ and after we apply :
$$
J^{\mathrm{Ising}}_{ij}(\sigma_i,\sigma_j) = J_{ij}(\sigma_i,\sigma_j) - \frac{1}{q}\sum_{\sigma_i} J_{ij}(\sigma_i,\sigma_j)  - \frac{1}{q} \sum_{\sigma_j} J_{ij}(\sigma_i,\sigma_j) + \frac{1}{q^2} \sum_{\sigma_i,\sigma_j} J_{ij}(\sigma_i,\sigma_j) 
$$
where q is the number of states for $\sigma$.

In [ ]:
%%cython --cplus --f
import numpy as np
import time
cimport cython
from libc.math cimport exp


cdef extern from "<random>" namespace "std":
    cdef cppclass mt19937:
        mt19937() # we need to define this constructor to stack allocate classes in Cython
        mt19937(unsigned int seed) # not worrying about matching the exact int type for seed
    
    cdef cppclass uniform_real_distribution[T]:
        uniform_real_distribution()
        uniform_real_distribution(T a, T b)
        T operator()(mt19937 gen) # ignore the possibility of using other classes for "gen"

    cdef cppclass uniform_int_distribution[T]:
        uniform_int_distribution()
        uniform_int_distribution(T a, T b)
        T operator()(mt19937 gen) # ignore the possibility of using other classes for "gen"


def test_uniform_distrs():
    """Small test for random number generation using https://www.cplusplus.com/reference/random/mt19937/"""
    cdef:
        mt19937 gen = mt19937(20)
        uniform_real_distribution[double] dist_real = uniform_real_distribution[double](0.0, 1.0)
        uniform_int_distribution[int] dist_int = uniform_int_distribution[int](0, 10)
     
    return dist_real(gen), dist_int(gen)


cdef class PottsMSA:
    cdef public double[:, ::1] field
    cdef public double[:, :, :, ::1] coupling
    cdef public int n_flips
    cdef public float temperature
    cdef public unsigned int seed
    cdef int n_states
    cdef int[:, :] nn_matrix
    cdef int n_vars

    def __init__(self, graph, double[:, ::1] field, double[:, :, :, ::1] coupling, float temperature, int n_flips):
        """
        Input:
        graph : networkx graph
        double[:,::1] field : h_i(\sigma_i)
        double[:,:,:,::1] coupling : J_{i,j}(\sigma_i,\sigma_j)
        float temperature 
        int n_flips : Number of flip to do 
        """
        self.n_states = np.intc(field.shape[1])
        self.temperature = temperature
        self.field = field
        self.coupling = coupling
        self.n_vars = np.intc(graph.number_of_nodes())
        self.n_flips = n_flips
        self._make_nn_matrix(graph)

    def _make_nn_matrix(self, graph):
        nn_matrix = -1 * np.ones((self.n_vars, self.n_vars), dtype=np.int32)
        for node in range(self.n_vars):
            neighbors = np.array([n for n in graph.neighbors(node)], dtype=np.int32)
            if len(neighbors) > 0:
                nn_matrix[node, :len(neighbors)] = neighbors
        self.nn_matrix = nn_matrix

    @cython.boundscheck(False)
    @cython.wraparound(False)
    @cython.initializedcheck(False)
    def generate(self, int n_sequences, seed=None, init=None):
        if init is None:
            init = np.random.randint(self.n_states, size=(n_sequences, self.n_vars), dtype=np.int8)
        if seed is None:
            self.seed = time.time()
        else:
            self.seed = seed
        cdef:
            int i
            char[:, ::1] init_view = init
            int _seed = self.seed
        self.MCMC(self.n_flips, init_view, _seed)

        return init

    @cython.boundscheck(False)
    @cython.wraparound(False)
    @cython.initializedcheck(False)
    def hamiltonian(self, char[::1] sequence):
        cdef:
            int i, j
            double h = 0
        for i in range(self.nn_matrix.shape[0]):
            h -= self.field[i, sequence[i]]
            for j in range(self.nn_matrix.shape[1]):
                if self.nn_matrix[i, j] == -1:
                    break
                elif self.nn_matrix[i, j] > i:  # Edge already included in Hamiltonian
                    h -= self.coupling[i,
                                       self.nn_matrix[i, j],
                                       sequence[i],
                                       sequence[self.nn_matrix[i, j]]]
        return h 

    @cython.boundscheck(False)
    @cython.wraparound(False)
    @cython.initializedcheck(False)
    cpdef void MCMC(self, int n_flips, char[:, ::1] init_view, int seed):  
        cdef:
            int selected_node, new_state
            int c_mutation
            double prob
            mt19937 gen = mt19937(seed)
            uniform_int_distribution[int] dist_states = uniform_int_distribution[int](0, self.n_states - 1)
            uniform_int_distribution[int] dist_vars = uniform_int_distribution[int](0, self.n_vars - 1)
            uniform_real_distribution[double] dist_prob = uniform_real_distribution[double](0.0, 1.0)

        for i in range(init_view.shape[0]):
            c_mutation = 0
            sequence = init_view[i]
            while c_mutation < n_flips:   
                selected_node = dist_vars(gen)
                new_state = dist_states(gen)
                if new_state >= sequence[selected_node]:
                    new_state += 1 
                prob = exp((1 / self.temperature) * 
                           (self.pseudo_hamiltonian(selected_node, new_state, sequence) -
                            self.pseudo_hamiltonian(selected_node, sequence[selected_node], sequence)))
                if dist_prob(gen) < prob:
                    sequence[selected_node] = new_state
                    c_mutation += 1

    @cython.boundscheck(False)
    @cython.wraparound(False)
    @cython.initializedcheck(False)
    cdef double pseudo_hamiltonian(self, int node, int state_node, char[::1] sequence):
        cdef:
            int i
            double h = 0
        h = self.field[node, state_node]
        for j in range(self.nn_matrix.shape[1]):
                if self.nn_matrix[node, j] == -1:
                    break
                else:
                    h += self.coupling[node,
                                       self.nn_matrix[node, j],
                                       state_node,
                                       sequence[self.nn_matrix[node, j]]]
        return h

## Generation of your MSA

In [ ]:
n_sequences = 100
# Set field, coupling, and temperature
J = np.diagflat([2] * 20) - 1
n_states = J.shape[0]
n_vars = graph.number_of_nodes()

coupling = np.array(np.broadcast_to(J, (n_vars, n_vars, n_states, n_states)),
                    order='C', dtype=float) 
field = np.zeros((n_vars, n_states), dtype=float)

temperature = 2

# Number of flips accepted to do:
n_flips = 4000  # You need to choose a good number of flips (You could look at the hamiltonian to choose)

# Creation of the class:
msa = PottsMSA(graph, field, coupling, temperature, n_flips)

In [ ]:
MSA = msa.generate(n_sequences)

### Bonus plot the hamiltonian to see when equilibrium is reached

In [ ]:
import numpy as np 

Number_average = 1
Delta_Flip = 100
Max_Flip = 10**5
LT = [0.5,1,2,3,4,5]

Class_MSA_Generation = Creation_MSA(graph, field, coupling, temperature, Delta_Flip)

L_energy = np.zeros((len(LT),Max_Flip//Delta_Flip))
L_energy_var = np.zeros((len(LT),Max_Flip//Delta_Flip))

for index_T, temperature in enumerate(LT):
    MSA = np.random.randint(Number_of_state , size = (Number_of_sequence_in_MSA, graph.number_of_nodes()),dtype = np.int8)
    print("T = " + str(temperature))
    Class_MSA_Generation.temperature = temperature
    for index_flip in range(Max_Flip//Delta_Flip):
        Halmitonian = [ Class_MSA_Generation.Hamiltonian(MSA[index_msa]) for index_msa in range(Number_of_sequence_in_MSA)]
        L_energy[index_T,index_flip] = np.mean(Halmitonian)
        Class_MSA_Generation.MSA_Equilibrium(1,MSA)



In [ ]:
L_flip = np.arange(Max_Flip//Delta_Flip)*Delta_Flip
for index_T, temperature in enumerate(LT):
    plt.plot(L_flip,L_energy[index_T],label = "T = %s"%temperature)
plt.xlabel("Mutation accepted")
plt.ylabel("Hamiltonian")
plt.title("J = %s" %J)
plt.semilogx()
plt.legend()
plt.show()